# Financial Analysis System with AgentNeo Integration

This Jupyter notebook demonstrates the integration of AgentNeo, a powerful tracing and monitoring tool, with a financial analysis system. AgentNeo provides seamless tracing capabilities for both function calls and AI model interactions, allowing for comprehensive analysis and debugging of complex systems.

## Setup and Imports

First, let's import the necessary libraries and set up our environment.

In [ ]:
import os
import random
from textblob import TextBlob
import openai
from dotenv import load_dotenv
from agentneo import AgentNeo, Tracer

# Load environment variables
load_dotenv("/path/to/your/.env")

# Initialize OpenAI API
openai.api_key = os.getenv("OPENAI_API_KEY")

## FinancialAnalysisSystem Class

Now, let's define our `FinancialAnalysisSystem` class with AgentNeo integration.

In [ ]:
# Initialize AgentNeo session
neo_session = AgentNeo(session_name="financial_analysis_session")

# Create project
neo_session.create_project(project_name="financial_analysis_project")

# Start tracing
tracer = Tracer(session=neo_session)
tracer.start()

In [ ]:
class FinancialAnalysisSystem:
    def __init__(self):
        self.stock_data = {}
        self.news_sentiment = {}
        self.economic_indicators = {}

    @tracer.trace_tool(name="fetch_stock_data")
    def fetch_stock_data(self, symbol):
        return {
            "symbol": symbol,
            "price": round(random.uniform(50, 500), 2),
            "change": round(random.uniform(-5, 5), 2),
        }

    @tracer.trace_tool(name="fetch_news_articles")
    def fetch_news_articles(self, company):
        return [
            f"{company} announces new product line",
            f"{company} reports quarterly earnings",
            f"{company} faces regulatory scrutiny",
        ]

    @tracer.trace_tool(name="analyze_sentiment")
    def analyze_sentiment(self, text):
        return TextBlob(text).sentiment.polarity

    @tracer.trace_tool(name="fetch_economic_indicators")
    def fetch_economic_indicators(self):
        return {
            "gdp_growth": round(random.uniform(-2, 5), 2),
            "unemployment_rate": round(random.uniform(3, 10), 2),
            "inflation_rate": round(random.uniform(0, 5), 2),
        }

    @tracer.trace_llm(name="analyze_market_conditions")
    def analyze_market_conditions(self, stock_data, sentiment, economic_indicators):
        prompt = f"""
        Analyze the following market conditions and provide a brief market outlook:
        Stock: {stock_data['symbol']} at ${stock_data['price']} (change: {stock_data['change']}%)
        News Sentiment: {sentiment}
        Economic Indicators:
        - GDP Growth: {economic_indicators['gdp_growth']}%
        - Unemployment Rate: {economic_indicators['unemployment_rate']}%
        - Inflation Rate: {economic_indicators['inflation_rate']}%
        """
        response = openai.chat.completions.create(
            model="gpt-4-0125-preview",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=150,
        )
        return response.choices[0].message.content.strip()

    @tracer.trace_llm(name="generate_investment_recommendation")
    def generate_investment_recommendation(self, market_outlook, risk_tolerance):
        prompt = f"""
        Based on the following market outlook and investor risk tolerance,
        provide a specific investment recommendation:
        Market Outlook: {market_outlook}
        Investor Risk Tolerance: {risk_tolerance}
        """
        response = openai.chat.completions.create(
            model="gpt-4-0125-preview",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=200,
        )
        return response.choices[0].message.content.strip()

    @tracer.trace_agent(name="FinancialAdvisorAgent")
    def financial_advisor_agent(self, stock_symbol, risk_tolerance):
        self.stock_data = self.fetch_stock_data(stock_symbol)
        news_articles = self.fetch_news_articles(stock_symbol)
        sentiment_scores = [self.analyze_sentiment(article) for article in news_articles]
        self.news_sentiment = sum(sentiment_scores) / len(sentiment_scores)
        self.economic_indicators = self.fetch_economic_indicators()
        market_outlook = self.analyze_market_conditions(
            self.stock_data, self.news_sentiment, self.economic_indicators
        )
        recommendation = self.generate_investment_recommendation(market_outlook, risk_tolerance)
        return recommendation

    def run_analysis(self, stock_symbol, risk_tolerance):
        recommendation = self.financial_advisor_agent(stock_symbol, risk_tolerance)
        print(f"\nAnalysis for {stock_symbol}:")
        print(f"Stock Data: {self.stock_data}")
        print(f"News Sentiment: {self.news_sentiment}")
        print(f"Economic Indicators: {self.economic_indicators}")
        print(f"\nInvestment Recommendation:\n{recommendation}")
        if "buy" in recommendation.lower():
            self.execute_buy_order(stock_symbol)
        elif "sell" in recommendation.lower():
            self.execute_sell_order(stock_symbol)
        else:
            print("No action taken based on the current recommendation.")

    @tracer.trace_tool(name="execute_buy_order")
    def execute_buy_order(self, symbol):
        print(f"Executing buy order for {symbol}")

    @tracer.trace_tool(name="execute_sell_order")
    def execute_sell_order(self, symbol):
        print(f"Executing sell order for {symbol}")

## Running the Analysis

Now let's create an instance of our `FinancialAnalysisSystem` and run an analysis.

In [ ]:
# Create an instance of FinancialAnalysisSystem
analysis_system = FinancialAnalysisSystem()

# Run an analysis for Apple stock with moderate risk tolerance
analysis_system.run_analysis("AAPL", "moderate")

# Stop the tracer when analysis is complete
tracer.stop()

## Analyzing the Results

After running the analysis, you can examine the output to see the stock data, news sentiment, economic indicators, and the investment recommendation. The AgentNeo tracer will have logged all the steps of the process, which you can later analyze using the AgentNeo dashboard.

To launch the AgentNeo dashboard and analyze the traces, you can use:

In [ ]:
from agentneo import launch_dashboard
launch_dashboard()

This will allow you to visualize the execution flow, identify any bottlenecks, and gain insights into the decision-making process of your financial analysis system.